<a href="https://colab.research.google.com/github/asheta66/Machine-Learning-2020/blob/main/NLP%20Programming/chapter_20/2_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Acquireing datasets
!git clone https://github.com/asheta66/Machine-Learning-2020
# Create Model via another notebook
%run Machine-Learning-2020/NLP\ Programming/chapter_20/1_prepare_text.ipynb

Cloning into 'Machine-Learning-2020'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 2852 (delta 189), reused 20 (delta 8), pack-reused 2569
Receiving objects: 100% (2852/2852), 29.12 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (512/512), done.
fatal: destination path 'Machine-Learning-2020' already exists and is not an empty directory.
/content/Machine-Learning-2020/Datasets/NLP
example.jpg              negative.txt        rhyme.txt  txt_sentoken/
metamorphosis_clean.txt  positive.txt        test.pkl   vocab.txt
mug.jpg                  republic_clean.txt  train.pkl
BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what
['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ar

### This Model will take aproximately 25 minutes to run with the GPU
### This Model will take aproximately **5-6 HOURS** to run without the GPU

In [2]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# define the model
def define_model(vocab_size, seq_length):
	model = Sequential()
	model.add(Embedding(vocab_size, 50, input_length=seq_length))
	model.add(LSTM(100, return_sequences=True))
	model.add(LSTM(100))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(vocab_size, activation='softmax'))
	# compile network
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize defined model
	model.summary()
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
# define model
model = define_model(vocab_size, seq_length)
# fit model
model.fit(X, y, batch_size=128, epochs=100)
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            370500    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
927/927 [==============================] - 21s 17ms/step - loss: 6.4909 - accuracy: 0.0623
Epoch 2/1